In [ ]:
# enabling the gpu
# install dependencies

import tensorflow as tf
from tensorflow import keras
import os
import tensorflow_datasets as tfds

# **AMAZON reviews to classify positive / negative reviews**

#**Using Tensorflow and Keras for the NLP classification**

#**First enabling the GPU and check our gpu property by the below code**

In [ ]:
!nvidia-smi

Thu Aug 27 14:35:17 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# checking the version

print(tf.__version__)

2.3.0


#**Loading the dataset from the Tensorflow datasets**

In [ ]:
# loading the datasets

datasets, info = tfds.load("amazon_us_reviews/Mobile_Electronics_v1_00", with_info= True)
 
# train datasets

train_data = datasets["train"]



/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Shuffling and writing examples to /root/tensorflow_datasets/amazon_us_reviews/Mobile_Electronics_v1_00/0.1.0.incompleteJJJ1U7/amazon_us_reviews-train.tfrecord


ERROR:absl:Statistics generation doesn't work for nested structures yet



Dataset amazon_us_reviews downloaded and prepared to /root/tensorflow_datasets/amazon_us_reviews/Mobile_Electronics_v1_00/0.1.0. Subsequent calls will reuse this data.


info - the information about the dataset (dataset exploring)

train - full dataset without split

if you run the below code
 you will see the dataset info and it contain every detail of the amazon's 
product

but, we need only reviews text and rating

In [ ]:
# info of the datasets

info


tfds.core.DatasetInfo(
    name='amazon_us_reviews',
    version=0.1.0,
    description='Amazon Customer Reviews (a.k.a. Product Reviews) is one of Amazons iconic products. In a period of over two decades since the first review in 1995, millions of Amazon customers have contributed over a hundred million reviews to express opinions and describe their experiences regarding products on the Amazon.com website. This makes Amazon Customer Reviews a rich source of information for academic researchers in the fields of Natural Language Processing (NLP), Information Retrieval (IR), and Machine Learning (ML), amongst others. Accordingly, we are releasing this data to further research in multiple disciplines related to understanding customer product experiences. Specifically, this dataset was constructed to represent a sample of customer evaluations and opinions, variation in the perception of a product across geographical regions, and promotional intent or bias in reviews.

Over 130+ million cus

 we are gonna take the review and star rating see the customer reviews


In [ ]:
# we are gonna take the review and star rating see the customer reviews

print(train_data)


<DatasetV1Adapter shapes: {data: {customer_id: (), helpful_votes: (), marketplace: (), product_category: (), product_id: (), product_parent: (), product_title: (), review_body: (), review_date: (), review_headline: (), review_id: (), star_rating: (), total_votes: (), verified_purchase: (), vine: ()}}, types: {data: {customer_id: tf.string, helpful_votes: tf.int32, marketplace: tf.string, product_category: tf.string, product_id: tf.string, product_parent: tf.string, product_title: tf.string, review_body: tf.string, review_date: tf.string, review_headline: tf.string, review_id: tf.string, star_rating: tf.int32, total_votes: tf.int32, verified_purchase: tf.int64, vine: tf.int64}}>


In [ ]:
# length of the datasets

len(list(train_data))


104975

In [ ]:
# setting the batch size and buffer size

BATCH_SIZE = 128
BUFFER_SIZE = 30000


#**Shuffle the datasets**

we don't need it to be ordered

In [ ]:
# getting the data shuffle

train_data = train_data.shuffle(BUFFER_SIZE, reshuffle_each_iteration= False)

 in tensorflow you need to iterate things to display ,for example


In [ ]:
# in tensorflow you need to iterate things to show for example

for reviews in train_data.take(2):
  print(reviews)
# all were in tensor shape

{'data': {'customer_id': <tf.Tensor: shape=(), dtype=string, numpy=b'21607088'>, 'helpful_votes': <tf.Tensor: shape=(), dtype=int32, numpy=2>, 'marketplace': <tf.Tensor: shape=(), dtype=string, numpy=b'US'>, 'product_category': <tf.Tensor: shape=(), dtype=string, numpy=b'Mobile_Electronics'>, 'product_id': <tf.Tensor: shape=(), dtype=string, numpy=b'B002R8CDMQ'>, 'product_parent': <tf.Tensor: shape=(), dtype=string, numpy=b'401037211'>, 'product_title': <tf.Tensor: shape=(), dtype=string, numpy=b'iPod Touch 2nd / 3rd Generation Silicone skin case cover 3G bundle, home / car charger, 8GB, 16GB, 32GB, 64GB (newest model) + Screen Guard Protector included!!!'>, 'review_body': <tf.Tensor: shape=(), dtype=string, numpy=b'I really am happy with the accessories for the car and house. The case broke right away and is poorly made.'>, 'review_date': <tf.Tensor: shape=(), dtype=string, numpy=b'2010-02-06'>, 'review_headline': <tf.Tensor: shape=(), dtype=string, numpy=b'ipod case'>, 'review_id': <

Extracting only the reviews text

In [ ]:
for reviews in train_data.take(4):
  # printing the reviews of people by iterating the tensorflow just like a json format

  print(reviews["data"]["review_body"])

tf.Tensor(b'I really am happy with the accessories for the car and house. The case broke right away and is poorly made.', shape=(), dtype=string)
tf.Tensor(b'The product was just as described and works well. The retractable USB is really usefull. I would buy it again.', shape=(), dtype=string)
tf.Tensor(b"I bought this armband solely because it was cheaper than the rest of armbands that are out there. I have to say, I am pleasantly surprised. I've had no trouble with it whatsoever. I go running with this armband all the time, and my iPod stays secure, and the armband doesn't slip. It's also easy to actually change my song selection and such while I'm running with the armband. I would definitely recommend this product to anyone who is looking for a no hassle, simply designed armband at a great price.", shape=(), dtype=string)
tf.Tensor(b'A no-frills, elegantly simple, truly seamless experience. I didn\'t have to worry about anything. I just got the product, took it out of the box, and s

To remove tensor shape use numpy()

Basically we are doing here is. showing the review texts and coorespondent rating

ratings like in range(1-5) but we want it to be 0 and 1

by making threshold as a 3 

0 - negative reviews  ( < 3 )

1 - positive reviews  ( > 3)

---



In [ ]:
# now making the rating with some threshold like positive and negative ( 0 and 1 )

for reviews in train_data.take(5):
  print(reviews["data"].get("review_body").numpy())
  
  print(reviews["data"].get("star_rating"))

  # here we are making the condition like 0 and 1

  print(tf.where(reviews["data"].get("star_rating")>3,1,0).numpy())

  print("\n\n")


b'I really am happy with the accessories for the car and house. The case broke right away and is poorly made.'
tf.Tensor(3, shape=(), dtype=int32)
0



b'The product was just as described and works well. The retractable USB is really usefull. I would buy it again.'
tf.Tensor(4, shape=(), dtype=int32)
1



b"I bought this armband solely because it was cheaper than the rest of armbands that are out there. I have to say, I am pleasantly surprised. I've had no trouble with it whatsoever. I go running with this armband all the time, and my iPod stays secure, and the armband doesn't slip. It's also easy to actually change my song selection and such while I'm running with the armband. I would definitely recommend this product to anyone who is looking for a no hassle, simply designed armband at a great price."
tf.Tensor(5, shape=(), dtype=int32)
1



b'A no-frills, elegantly simple, truly seamless experience. I didn\'t have to worry about anything. I just got the product, took it out of the bo

# **Data Preprocessing**

going to tokenize the words in this review

In [ ]:
# now tokenizing

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences



In [ ]:
# considering the vocab_size

vocab_size = 73738

creating a sentences and label list and appending the reviews texts and rating

sentences ----> reviews text

labels -------> ratings( 0 and 1)

In [ ]:
def sep_text_reviews(train_data):
  sentences = []
  labels = []
  for reviews in train_data:
    sentences.append(reviews["data"].get("review_body").numpy().decode("utf-8"))
    labels.append(tf.where(reviews["data"].get("star_rating")>3,1,0).numpy())
  return sentences, labels

# call the function

sentences, labels = sep_text_reviews(train_data)


In [ ]:
len(labels)

104975

In [ ]:
len(sentences)

104975

now all the reviews and ratings are stored in a list

In [ ]:
for i in range(0,3):
  print(sentences[i])
  print(labels[i])
  print("\n\n")

I really am happy with the accessories for the car and house. The case broke right away and is poorly made.
0



The product was just as described and works well. The retractable USB is really usefull. I would buy it again.
1



I bought this armband solely because it was cheaper than the rest of armbands that are out there. I have to say, I am pleasantly surprised. I've had no trouble with it whatsoever. I go running with this armband all the time, and my iPod stays secure, and the armband doesn't slip. It's also easy to actually change my song selection and such while I'm running with the armband. I would definitely recommend this product to anyone who is looking for a no hassle, simply designed armband at a great price.
1





#**Train and Test split**

from the length of datasset were 104975

splitting into , 

training ---> 85000

testing ----> 104975 - 85000

In [ ]:
import numpy as np

training_size = 85000

training_data =  np.array(sentences[0:training_size])
training_labels = np.array(labels[0:training_size])

testing_data = np.array(sentences[training_size:])
testing_labels = np.array(labels[training_size:])

training_data.shape , training_labels.shape, testing_data.shape, testing_labels.shape

((85000,), (85000,), (19975,), (19975,))

Tokenizing the reviews into tokenize word

In [ ]:
tokenizer = Tokenizer(num_words= 73738 , oov_token= "<OOV>")
tokenizer.fit_on_texts(training_data)

word_index = tokenizer.word_index


From the below code,

as you see the reviews text words are numbered 

In [ ]:
word_index.items()

dict_items([('<OOV>', 1), ('the', 2), ('i', 3), ('it', 4), ('and', 5), ('to', 6), ('a', 7), ('is', 8), ('this', 9), ('for', 10), ('my', 11), ('of', 12), ('in', 13), ('br', 14), ('that', 15), ('with', 16), ('on', 17), ('not', 18), ('you', 19), ('but', 20), ('was', 21), ('have', 22), ('as', 23), ('so', 24), ('very', 25), ('great', 26), ('one', 27), ('be', 28), ('case', 29), ('good', 30), ('if', 31), ('are', 32), ('product', 33), ('or', 34), ('would', 35), ('just', 36), ('when', 37), ('all', 38), ('at', 39), ('use', 40), ('from', 41), ('like', 42), ('had', 43), ('they', 44), ('out', 45), ('can', 46), ('will', 47), ('no', 48), ('an', 49), ("it's", 50), ('well', 51), ('up', 52), ('ipod', 53), ('your', 54), ('works', 55), ('get', 56), ('has', 57), ('only', 58), ('me', 59), ('work', 60), ('sound', 61), ('what', 62), ('time', 63), ('than', 64), ('price', 65), ('about', 66), ('screen', 67), ('quality', 68), ('more', 69), ('really', 70), ('there', 71), ('does', 72), ('do', 73), ('bought', 74), (

based on the sentences we are using the numbered reviews words

In [ ]:
training_sequences = tokenizer.texts_to_sequences(training_data)

In [ ]:

training_sequences[1]

[2,
 33,
 21,
 36,
 23,
 579,
 5,
 55,
 51,
 2,
 2114,
 144,
 8,
 70,
 6350,
 3,
 35,
 88,
 4,
 164]

we need a maximum length of the reviews overall

In [ ]:
max_sequence_len = max([len(x) for x in training_sequences])
max_sequence_len

2917

padding is convert our sentences with the maximum length of reviews 

eg:
 
 our reviews maximum length is 2943

 so, converting every sentence with that maximum length by adding 0 in front

In [ ]:
training_padded = pad_sequences( training_sequences, maxlen= max_sequence_len, truncating = "post")

In [ ]:
training_padded[1]

array([  0,   0,   0, ...,  88,   4, 164], dtype=int32)

Similar way with the testing sentences

In [ ]:
testing_sequences = tokenizer.texts_to_sequences(testing_data)
testing_padded = pad_sequences(testing_sequences, maxlen= max_sequence_len)

#**Define our model**

- we are using bidirectional LSTM for our model [link for the Documentation](https://keras.io/api/layers/recurrent_layers/lstm)

In [ ]:
# model define


model = keras.Sequential([
                          
keras.layers.Embedding(vocab_size, 16, input_length= max_sequence_len),
 keras.layers.Bidirectional(keras.layers.LSTM(32)),
keras.layers.Dense(24, activation='relu'),
keras.layers.Dense(1, activation='sigmoid')])

model.compile(
    loss="binary_crossentropy",
    optimizer = "adam",
    metrics = ["accuracy"]
)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 2917, 16)          1179808   
_________________________________________________________________
bidirectional (Bidirectional (None, 64)                12544     
_________________________________________________________________
dense (Dense)                (None, 24)                1560      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 25        
Total params: 1,193,937
Trainable params: 1,193,937
Non-trainable params: 0
_________________________________________________________________


#**Training our model**

- if it start training, 

- so do you have any plan for next 30 minutes just go for it and came back later

In [ ]:
model.fit(training_padded, training_labels, epochs = 12, batch_size = 128, validation_data= (testing_padded, testing_labels))

Epoch 1/12
665/665 [==============================] - 378s 568ms/step - loss: 0.3622 - accuracy: 0.8414 - val_loss: 0.2965 - val_accuracy: 0.8805
Epoch 2/12
665/665 [==============================] - 376s 565ms/step - loss: 0.2563 - accuracy: 0.8970 - val_loss: 0.2868 - val_accuracy: 0.8818
Epoch 3/12
665/665 [==============================] - 373s 560ms/step - loss: 0.2194 - accuracy: 0.9140 - val_loss: 0.2968 - val_accuracy: 0.8842
Epoch 4/12
665/665 [==============================] - 373s 560ms/step - loss: 0.1900 - accuracy: 0.9271 - val_loss: 0.3473 - val_accuracy: 0.8771
Epoch 5/12
468/665 [====================>.........] - ETA: 1:41 - loss: 0.1631 - accuracy: 0.9391

our model is overfitting the data but our validation accuracy increasing  yeah that's good sign

if you have more time just increase the epochs values ( 15, 20, 50).....


#**Save the model**

In [ ]:
model.save("amazon_reviews.h5")

#**Loading our model**

well i try to give another notebook for loading our saved model

because to reduce the training time 

(if you want to train it just go for it and try to increase the validation accuracy )



In [ ]:
model_path = "/content/amazon_reviews.h5"

In [ ]:
from keras.models import load_model

In [ ]:
model = load_model(model_path)

In [ ]:
model.summary()

#**Predicting the reviews**

for prediction , 

- first thing we need a tokenizer to convert the output text into vector(numbered)

- we need to padding it and then going for prediction

In [ ]:
def prediction(model, text):
  sequences = tokenizer.texts_to_sequences([text])
  padded = pad_sequences(sequences, maxlen = max_sequence_len, truncating = "post")
  reviews = model.predict(padded)

  if reviews[0]>0.5:
    print("it is a positive percent : ",str(reviews[0][0]*100)[:3])
  else:
    print("it is a negative percent : ",str(reviews[0][0]*100)[:3])





now calling the function , with output text


In [ ]:
text = "recently i buyed a laptop which battery were good"
prediction(model, text)

wow, positive reviews and lets make it complicated output text

changing  good ---> drained

In [ ]:
text = " waste product very much disappointing"
prediction(model, text)